# Drift Watch Monitoring Server

Drift Watch monitoring server is a stand-alone HTTP server that serves multiple REST API endpoints for storing experiments and drifts. While Drift Watch can be used in local environment, for starters it is recommended to use one of the available online endpoints. This way you can easily share your results with your team members and access them from anywhere:

**Collaboration**: Multiple users can create experiment in the same endpoints where to log drifts.

**Sharing Results**: The monitoring server also serves as monitoring interface, where members can easily explore each other’s results.


## Logging to a Monitoring Server


Once you started the tracking server, you can connect your local clients by set the **DRIFT_MONITOR_URL** environment variable to the server’s URI, along with its scheme and port (for example, http://localhost:5000)


In [1]:
import os

os.environ["DRIFT_MONITOR_URL"] = "https://drift-watch.dev.ai4eosc.eu"
os.environ["DRIFT_MONITOR_MYTOKEN"] = "token_from_mytoken.data.kit.edu"

In [2]:
from uuid import uuid4

from drift_monitor import DriftMonitor, new_experiment, register

Register to the service in case you did not do it before.


In [3]:
register(accept_terms=True)

Define your detector methods. In this example, for the aim os simplicity, we mock detectors where a tuple (detection, features) is returned. The detection is a boolean indicating if the detector found an anomaly or not. The features is a dictionary with the features used in the detector.


In [4]:
def concept_detector(*args, **kwds) -> tuple[bool, dict]:
    return True, {"feature1": 0.05, "feature2": 0.1}


def data_detector(*args, **kwds) -> tuple[bool, dict]:
    return True, {"feature3": 0.02, "feature4": 0.08}

Now open a DriftMonitor context and call the detectors. The DriftMonitor will store the results in a dictionary with the detector names as keys and the tuple (detection, features) as values.


In [5]:
experiment_name = f"experiment_{uuid4()}"
description = "Test experiment example"
new_experiment(experiment_name, description, public=True)

{'created_at': '2024-10-22T14:40:35.270588',
 'description': 'Test experiment example',
 'id': 'd4acb385-9014-4e1a-96bf-ed95d38e10b4',
 'name': 'experiment_c8036cb5-7c51-4c8c-b69f-38bb4d030821',
 'permissions': {'47f61d0e-87c3-47cb-8fe3-37526d3b66de': 'Manage'},
 'public': True}

In [6]:
with DriftMonitor(experiment_name, "model_1") as monitor:
    detected, detection_parameters = concept_detector()
    monitor.concept(detected, detection_parameters)
    detected, detection_parameters = data_detector()
    monitor.data(detected, detection_parameters)

When the context is closed, the DriftMonitor will send the results to the API with status "Completed". If and exception occurs, the DriftMonitor will catch the exception, send the results with status "Failed" and re-raise the exception.
